# Демонстрация инструмента последовательного снятия снимков агрегированных данных из потоковых данных

Импортируем необходимые библиотеки.

In [2]:
import subprocess
import time

import requests
import IPython.display

import constants
from _development.tests.test_utilities import print_response, print_tables_side_by_side, terminate_shell_subprocess

Запустим генератор потоковых данных.

Потоковые данные содержат 3 поля:
- a: int
- b: int
- c: float

Генератор потоковых данных работает по следующему алгоритму:
1. Добавление 10 новых объектов, где:
    - a – случайное целое число от 0 до 10
    - b – случайное целое число от 0 до 20
    - с – случайное число от 0 до 10
2. Обновление 1 случайного объекта:
    - a задается равным 1000
    - b и c умножаются на 10
3. Удаление 1 случайного объекта

Данные изменения применяются к таблице базы данных PostgreSQL. Оттуда данные выгружаются в Apache Kafka инструментом CDC Debezium. Далее данные поступают в разработанный инструмент.

In [6]:
stream_generator = subprocess.Popen('python stream_generator.py', shell=True)

Запустим разработанный инструмент.

In [7]:
main = subprocess.Popen('python ../../main.py', shell=True)

Зададим имя топика Apache Kafka, откуда инструмент будет получать данные.

In [8]:
topic_name = 'postgres_source.public.demo_table'

Подготовим JSON запроса подключения к источнику данных CREATE SOURCE.

In [9]:
create_source = {'query_type': 'CREATE SOURCE',
                 'name': 'demo_source', 'type': 'DebeziumSource',
                 'parameters': {'kafka_topic_name': topic_name, 'group_id': None, 'auto_offset_reset': 'earliest',
                                'bootstrap_servers': ['kafka:9092'], 'consumer_timeout_ms': 1000}}

Подготовим JSON запроса создания материализованного представления CREATE MATERIALIZED VIEW.



In [22]:
create_view = {'query_type': 'CREATE MATERIALIZED VIEW',
               'name': 'demo_view', 'view_source_name': 'demo_source', 'groupby_columns': ['a'],
               'parameters': {'extrapolation': True, 'extrapolation_method': 'linear'},
               'aggregates': [{'function': 'Sum', 'column': 'b'}, {'function': 'Avg', 'column': 'c'}, {'function': 'CountDistinct', 'column': 'b'},
                              {'function': 'CountDistinctCBF', 'column': 'b', 'parameters': {'expected_element_count': 100, 'false_positive_probability': 0.01}}]}

Подготовим JSON запроса получения данных SELECT.

In [11]:
select = {'query_type': 'SELECT', 'name': 'demo_view', 'orderby': [['a', 'DESC']], 'format': 'tabulate'}

In [12]:
select_extrapolated = {'query_type': 'SELECT EXTRAPOLATED', 'name': 'demo_view', 'orderby': [['a', 'DESC']], 'format': 'tabulate'}

In [13]:
print_response(requests.post(f'http://localhost:{constants.SERVER_PORT}', json=create_source))

200 OK


In [23]:
print_response(requests.post(f'http://localhost:{constants.SERVER_PORT}', json=create_view))

400 View with name demo_view already exists


In [15]:
while True:
    select_response = requests.post(f'http://localhost:{constants.SERVER_PORT}', json=select).content.decode()
    select_extrapolated_response = requests.post(f'http://localhost:{constants.SERVER_PORT}', json=select_extrapolated).content.decode()
    print_tables_side_by_side(select_response, select_extrapolated_response, 'SELECT', 'SELECT EXTRAPOLATED')
    time.sleep(5)
    IPython.display.clear_output()

KeyboardInterrupt: 

In [12]:
terminate_shell_subprocess(main)

In [21]:
terminate_shell_subprocess(stream_generator)

In [4]:
import kafka_utilities

kafka_utilities.delete_topic(topic_name)